## Prerequisites

### Completed the Getting Started Notebook

In order for you to start creating templates in QuickSight you first must have hydrated your data lake and have data sitting in your S3 data lake. Following the steps in the `Getting_Started_With_AMC_Quickstart.ipynb` Notebook will lead you through the steps to do so.

### Sign Up For Quicksight

To subscribe to QuickSight:

1. Sign in to your AWS account and open QuickSight from the AWS Management Console. You can find it under Analytics or by searching for QuickSight.

2. Choose Sign up for QuickSight.

3. Choose Standard or Enterprise Edition.

4. For both Standard and Enterprise editions, make choices for the following items:

    - Enter a unique account name for QuickSight. Your account name can only contain characters (A–Z and a–z), digits (0–9), and hyphens (-). If you use Microsoft AD, and it has a default alias, this alias is used for the account name.

    - Enter a notification email address for the QuickSight account owner or group. This email address receives service and usage notifications.
    
    - Ensure Amazon S3, Amazon Athena, and Amazon SageMaker all have QuickSight Access

5. Review the choices that you made, then choose Finish.


## Import Libraries and Helper Functions

In [ ]:
import boto3
import awswrangler as wr
from quicksight_helper_functions.quicksight_functions import QSHelperFunctions

In [ ]:
client = boto3.client('quicksight')
client_lf = boto3.client('lakeformation')
qs = QSHelperFunctions()

### Set QuickSight Configuration

1. Enter the desired names for the following QuickSight Variables:
     - Data Source Name
     - Template Name
     - Analysis Name
     - Dashboard Name
2. Change the Glue DB and Table Names to your particular Glue Database and Table Created
3. Ensure that the:
    - Template ARN is the ARN of the Public Template Provided by AMC
    - The Dataset Name matches the Dataset used for the Public Templated Provided by AMC
 

In [ ]:
#Set QuickSight Variables 
data_source_name="amc_data_source_test"
dataset_name="amc_data_set"
template_name = "amc_template"
analysis_name = "amc_time_to_conversion_analysis"
dashboard_name="amc_dataset_dashboard"

#CHANGE THESE TO YOUR GLUE DB and Table Name
glue_db="<ENTER YOUR GLUE DB NAME HERE>"  
glue_table="<ENTER YOUR GLUE TABLE NAME HERE>"

region="us-east-1" #CHANGE TO YOUR AWS REGION
accountid=wr.sts.get_account_id()

## Add LakeFormation Permission

Before creating any Quicksight Assets, we must pass AWS Lake Formation Permissions to both the QuickSight service role and your QuickSight User in order to get permissions to access the data.

#### Add LakeFormation Permission To The Default QuickSight Service Role

In [ ]:
#Add admin role to lakeformation with SELECT permissions
admin_role_arn_if = f'arn:aws:iam::{accountid}:role/service-role/aws-quicksight-service-role-v0'
qs.grant_permission_lf(admin_role_arn_if, glue_db, ["SELECT"])

#### Add LakeFormation Permission To Your QuickSight User

Run the below cell to list all the users in your QuickSight account:

In [55]:
users=[]
for user_details in wr.quicksight.list_users():
    users.append(user_details["UserName"])
users

['ADMIN/noahpaig-Isengard']

**Copy your Username in the below cell and run the following 2 cells to give your QuickSight User LakeFormation Permissions To Your Glue DB**

In [ ]:
# Add Username to Grant Permissions To
username = "<ENTER YOUR USERNAME HERE>"
username_list=[username]

In [ ]:
#Add quicksight admin user to lakeformation with SUPER permissions
quicksight_arn_lf = f'arn:aws:quicksight:{region}:{accountid}:user/default/{username}'
# quicksight_arn_lf = "arn:aws:quicksight:us-east-1:554934668058:user/default/ADMIN/noahpaig-Isengard"
qs.grant_permission_lf(quicksight_arn_lf, glue_db, ["ALL"])

## Creating Amazon QuickSight Resources

Now begin creating Amazon QuickSight resources by running the following cells

### Check if QuickSight Dashboard Exists (Move Forward Only If Not)

First check to make sure the dashboard name your are trying to create exists or not. If it already exists you will NOT be able to create another dashboard with the same name. Either delete the current dashboard with the given name or change `dashboard_name` to a new value


In [ ]:
qs.check_dashboard_exists(dashboard_name)      

### Create QuickSight Data Source and Data Set For Athena Source

Run these 2 cells to create an Athena Data Source in QuickSight. We also create a data set from the Glue DB and Table we specified earlier:

In [ ]:
qs.create_data_source(data_source_name, username_list)

In [ ]:
qs.create_data_set(dataset_name, glue_db, glue_table, data_source_name, username_list)

### Check to see if you have a QuickSight Template in your QuickSight Account

The following cell lists all of the QuickSight templates in your AWS Account, if any exist. A template is required in your AWS Account in order to create continue using this notebook and create a QuickSight Dashboard or QuickSight Analysis:

In [ ]:
templates = wr.quicksight.list_templates()
templates

**Copy the template name in the cell below. This template will be used to create a QuickSight Dashboard and/or Analysis in the following steps.**

In [ ]:
template_name = "<ENTER TEMPLATE NAME>"

### Option 1: Create and Publish Quicksight Dashboard from A Template

Run the below cell to use the template in your AWS Account to create a QuickSight Dashboard. 

_NOTE: In this case you can not change the underlying analysis from the QuickSight Template (i.e. the dashboard can not be altered)_

In [ ]:
qs.create_dashboard_from_template(dashboard_name, username_list, region, dataset_name, template_name, accountid)

### Option 2: Create A QuickSight Analysis from A Template

Run the below cell to use the template in your AWS Account to create a QuickSight Analysis. In this case, you can replicate the underlying analysis, verify or review, and develop further.

_NOTE: To publish this analysis you must go click on the analysis created in QuickSight, then publish to a QuickSight Dashboard by clicking on `Share` and `Publish as Dashboard`_

In [ ]:
qs.create_analysis_from_template(accountid, analysis_name, username_list, dataset_name, region, template_name)

## Delete QuickSight Resources

The below cells will help clean up any unused resources in QuickSight you would like to delete

#### Delete A Single Dashboard

Pass the `dashboard_name` to be deleted and run the below cell to delete that QuickSight Dashboard:

In [ ]:
qs.delete_quicksight_dashboard(dashboard_name)

#### Delete A Single Analysis
Pass the `analysis_name` to be deleted and run the below cell to delete that QuickSight Analysis:

In [ ]:
qs.delete_quicksight_analysis(accountid, analysis_name, region)

#### Delete A Single Template
Pass the `template_name` to be deleted and run the below cell to delete that QuickSight Template:

In [ ]:
qs.delete_quicksight_template(template_name)

#### Delete A Single Data Set
Pass the `dataset_name` to be deleted and run the below cell to delete that QuickSight Dataset:

In [ ]:
qs.delete_quicksight_dataset(dataset_name)

#### Delete A Single DataSource
Pass the `data_source_name` to be deleted and run the below cell to delete that QuickSight Data Source:

In [ ]:
qs.delete_quicksight_data_source(data_source_name)

#### OPTIONAL: Delete ALL QuickSight Datasets, Data Sources, Templates, and Dashboards.

If you would like to delete all of the resources you have in QuickSight (excluding QuickSight Analyses), run the below cell:

_NOTE: This will also delete all of the sample Datasets that QuickSight Provides to users when they first subscribe to Amazon QuickSight_

In [ ]:
qs.delete_all_quicksight_datasets()
qs.delete_all_quicksight_data_sources()
qs.delete_all_quicksight_templates()
qs.delete_all_quicksight_dashboards()

_NOTE: In order to delete all of the QuickSight Analyses in your account, either pass the names of the analyses to the `qs.delete_quicksight_analysis()` function or manual delete the analyses from your QuickSight console_

### --------------------------------------------------------------------

## EXAMPLE: Grant or Revoke Another Quicksight User Permissions

The Below Cell is used to add another quicksight user permissions or admin permissions to a QuickSight Dashboard.


In [ ]:

username_list=["<ENTER YOUR QUICKSIGHT USERNAME OR A LIST OF USERNAMES>"]
dashboard_user_type = "DASHBOARD_USER" #OR "DASHBOARD_ADMIN"
permission_type = "GRANT" #OR "REVOKE"

qs.update_dashboard_permissions(permission_type, dashboard_name, username_list, dashboard_user_type, region)